In [2]:
import pandas as pd

In [3]:
dataset = pd.read_csv('moviedataset.csv' , encoding='cp437')

In [4]:
dataset = dataset[['Title','Genre','Director','Actors','Plot']]
dataset.tail()

,Title,Genre,Director,Actors,Plot
245,The Lost Weekend,"Drama, Film-Noir",Billy Wilder,"Ray Milland, Jane Wyman, Phillip Terry, Howard...",The desperate life of a chronic alcoholic is f...
246,Short Term 12,Drama,Destin Daniel Cretton,"Brie Larson, John Gallagher Jr., Stephanie Bea...",A 20-something supervising staff member of a r...
247,His Girl Friday,"Comedy, Drama, Romance",Howard Hawks,"Cary Grant, Rosalind Russell, Ralph Bellamy, G...",A newspaper editor uses every trick in the boo...
248,The Straight Story,"Biography, Drama",David Lynch,"Sissy Spacek, Jane Galloway Heitz, Joseph A. C...",An old man makes a long journey by lawn-mover ...
249,Slumdog Millionaire,Drama,"Danny Boyle, Loveleen Tandan","Dev Patel, Saurabh Shukla, Anil Kapoor, Raj Zu...",A Mumbai teen reflects on his upbringing in th...


In [5]:
import re
import nltk
#nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
corpus=[]
for i in range(249):
    row=dataset['Genre'][i]+dataset['Director'][i]+dataset['Actors'][i]+dataset['Plot'][i]
    review = re.sub('[^a-zA-Z]',' ',row)
    review=review.lower()
    review=review.split()
    ps=PorterStemmer()
    review=[word for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)
    dataset['Plot'][i]=review
dataset=dataset[['Title','Plot']]
dataset.head()

,Title,Plot
0,The Shawshank Redemption,crime dramafrank darabonttim robbins morgan fr...
1,The Godfather,crime dramafrancis ford coppolamarlon brando a...
2,The Godfather: Part II,crime dramafrancis ford coppolaal pacino rober...
3,The Dark Knight,action crime dramachristopher nolanchristian b...
4,12 Angry Men,crime dramasidney lumetmartin balsam john fied...


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
cv=CountVectorizer()
matrix=cv.fit_transform(dataset['Plot'])
cosine_sim = cosine_similarity(matrix, matrix)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=15,min_df=0., max_df=1.0)
print(cv.fit_transform(dataset['Plot']))
from pandas import DataFrame
x=cv.fit_transform(dataset['Plot']).toarray()
print(DataFrame(x, columns=cv.get_feature_names()).to_string())

  (0, 4)	1
  (1, 4)	2
  (2, 4)	2
  (2, 11)	2
  (2, 9)	1
  (2, 6)	1
  (3, 4)	1
  (3, 0)	1
  (4, 4)	1
  (4, 8)	1
  (5, 2)	1
  (5, 5)	1
  (5, 13)	1
  (6, 5)	1
  (6, 1)	1
  (7, 4)	1
  (7, 8)	1
  (8, 9)	1
  (9, 5)	1
  (9, 1)	1
  (10, 5)	1
  (10, 3)	1
  (11, 0)	1
  (11, 1)	1
  (12, 0)	1
  :	:
  (237, 1)	1
  (238, 6)	1
  (239, 5)	1
  (239, 13)	1
  (240, 0)	1
  (240, 1)	1
  (241, 11)	1
  (241, 0)	1
  (241, 1)	1
  (242, 8)	1
  (242, 1)	1
  (243, 9)	1
  (243, 5)	1
  (243, 1)	1
  (244, 2)	1
  (244, 5)	1
  (244, 13)	1
  (244, 14)	1
  (245, 9)	1
  (245, 5)	1
  (246, 8)	1
  (247, 5)	1
  (247, 3)	1
  (248, 2)	1
  (248, 10)	1
     action  adventure  biography  comedy  crime  drama  family  help  john  life  man  robert  sci  war  young
0         0          0          0       0      1      0       0     0     0     0    0       0    0    0      0
1         0          0          0       0      2      0       0     0     0     0    0       0    0    0      0
2         0          0          0       0     

In [8]:
indices = pd.Series(dataset.index)
print(indices.index[0])
def recommendations(title, cosine_sim = cosine_sim):
    for i in range(0,250):
        if dataset['Title'][i]==title:
            title=i
    # initializing the empty list of recommended movies
    recommended_movies = []
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(dataset.index)[i])
        
    return recommended_movies

0


In [9]:
a=recommendations("Slumdog Millionaire")

In [10]:
for i in range(0,250):
    if i in a:
        print(dataset['Title'][i])

Goodfellas
The Green Mile
The Kid
Amadeus
To Kill a Mockingbird
Some Like It Hot
Batman Begins
Up
The Big Lebowski
Lion
